![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
<p style="text-align: center; padding:20px;"><br />
  <span style="font-size: 80px; font-weight:800;">SmartCart</span><br /><br />
  <span style="font-size: 40px; font-weight:500; line-height: 1.2;"><br />
      A Generative <br />
      AI-powered Product Recommendation Engine
  </span>
    <span style="font-size: 30px; font-weight:500; line-height: 1.2;"><br />
      <br />By <br /><br />
      WatsonX_Team_007
  </span>
</p>

This notebook contains steps and code to demonstrate how better we can improve product recommendation based off client reviews and factors like seasons, and celebrity influences.

## Data Sources
Economic Data Source:<br />
https://databank.worldbank.org/source/world-development-indicators

Other Secondary Data Source: <br />
https://www.kaggle.com/datasets/vivekgediya/ecommerce-product-review-data<br />
https://www.kaggle.com/datasets/kamganganthony/the-android-app-market-on-google-play/data<br />

Celebrity Data Source:<br />
https://robbreport.com/<br />
https://www.thecut.com<br />
https://www.thefashionisto.com/<br />
https://www.refinery29.com/en-us/2022/09/11133893/claire-sulmers-fashion-bomb-daily-interview<br />

## watsonx API connection 

In [1]:
import os, types
import requests
import getpass
import ibm_boto3
import pandas as pd
from pathlib import Path
from botocore.client import Config
from ibm_watsonx_ai.foundation_models import Model

## Defining Model Credentials

In [2]:
def get_credentials():
	return {
		"url" : "https://us-south.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("Please enter your api key (hit enter): ")
	}

## Defining Foundational Model ID

In [3]:
model_id = "ibm/granite-20b-multilingual"

## Defining the Model Parameters

In [4]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 900,
    "repetition_penalty": 1.05
}

## Setting Project and Space ID

In [5]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")
print(space_id)

None


## Defining the Model Object

In [6]:
model = Model(
	model_id = model_id,
	params = parameters,
	credentials = get_credentials(),
	project_id = project_id,
	space_id = space_id
	)

Please enter your api key (hit enter): ········


## Defining the inferencing input for chat

In [7]:
# Initialize the IBM COS client
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id={Your_API_Key},
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us-south.cloud-object-storage.appdomain.cloud')

## Defining Function Scopes

In [8]:
def load_cos_files(bucket, object_keys, max_size=1000):
    documents = []
    
    for object_key in object_keys:
        # Fetch each file
        streaming_body = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']
        
        # Read and decode the content
        file_content = streaming_body.read().decode('utf-8')
        
        # Trim the content if it exceeds max_size, Limit to first 1000 characters
        trimmed_content = file_content[:max_size] 
        documents.append(trimmed_content)
    
    # Combine all file contents into a single string
    return "\n".join(documents)

# Weather API call
def get_weather_data(city, api_key):
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        'q': city,
        'appid': api_key,
        'units': 'metric'  # For temperature in Celsius
    }
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        weather_data = response.json()
        return {
            'temperature': weather_data['main']['temp'],
            'description': weather_data['weather'][0]['description'],
            'city': weather_data['name']
        }
    else:
        return None

## Loading Files from COS for Model Grounding

In [9]:
bucket = 'Your_bucket_address'
object_keys = [
    'Flipkart_Reviews - Electronics.txt',
    'Product Review Data.txt',
    'Magazine_Subscriptions.txt',
    'economic.txt',
    'celebrity_info.txt'
]
combined_documents = load_cos_files(bucket, object_keys)

## Calling Weather API Function

In [10]:
api_key = 'Your_Openweather_API_key'
city = input("City: ")
weather_info = get_weather_data(city, api_key)

if weather_info:
    weather_summary = (f"Weather in {weather_info['city']}:\n"
                       f"Temperature: {weather_info['temperature']}°C\n"
                       f"Description: {weather_info['description']}\n")
    
    # Combine weather information with the documents
    combined_documents_with_weather = f"{combined_documents}\n\n{weather_summary}"
else:
    combined_documents_with_weather = f"{combined_documents}\n\nWeather information could not be retrieved."

print(weather_summary)

City: Port Harcourt
Weather in Port Harcourt:
Temperature: 31.1°C
Description: light rain



## Prompt Feature

In [13]:
# Get user input
question = input("Question: ")
formattedQuestion = f"""<|user|>\n{question}\n<|assistant|>\n"""

# Learn from data
prompt_input = f"Grounding documents:\n{combined_documents_with_weather}\n"
prompt = f"{prompt_input}{formattedQuestion}"

# Generate the response
generated_response = model.generate_text(prompt=prompt, guardrails=True)
print(f"AI: {generated_response}")

Question: Considering the products review performance, how do you think the economic factors and celebrity lifestyle can influence the regular person's buying pattern?
AI: The review performance of products can significantly impact the buying patterns of regular people, as it often reflects the real-world experience of using the product. Economic factors, such as inflation and GDP deflator, can also affect purchasing decisions, as they represent the overall health of the economy and the cost of living. When inflation is high, the cost of goods and services increases, which may lead people to prioritize essential items over non-essential ones. Additionally, a positive GDP deflator indicates economic growth, which can boost consumer confidence and lead to increased spending.

Celebrity lifestyle, on the other hand, can sometimes influence buying patterns indirectly. If a celebrity endorses or promotes a product, it can create a sense of aspiration or desirability, making people feel like

## Sample Questions

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2023 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  